In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.applications.vgg19 import VGG19
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
import pickle as pkl
import datetime

# Load Cifar10 dataset

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
print('The training set has {0} samples with dimentions {1}.'.format(x_train.shape[0], x_train.shape[1:]))
print('The test set has {0} samples with dimentions {1}.'.format(x_test.shape[0], x_test.shape[1:]))

The training set has 50000 samples with dimentions (32, 32, 3).
The test set has 10000 samples with dimentions (32, 32, 3).


In [22]:
NUM_CLASSES = 10
IMG_ROWS, IMG_COLS = 32, 32
BATCH_SIZE = 512
EPOCHS = 20
VAL_SPLIT = .2
LEARNING_RATE = 1e-2
SGD_MOMENTUM = .9

# Preprocess data

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [7]:
x_train -= np.mean(x_train)
x_test -= np.mean(x_test)

In [8]:
x_train /= np.std(x_train)
x_test /= np.std(x_test)

In [9]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

# Build model

In [10]:
input_tensor = Input(shape=(IMG_ROWS, IMG_COLS, 3))

In [11]:
base_model = VGG19(include_top=False, input_tensor=input_tensor)

In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [14]:
vgg19 = Model(inputs=base_model.input, outputs=predictions)

In [15]:
vgg19.compile(optimizer=SGD(lr=LEARNING_RATE, momentum=SGD_MOMENTUM), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model

In [23]:
now = datetime.datetime.now()
tensorflow_log_dir = "Graph/" + now.strftime("%Y%m%d-%H%M%S") + "/"

In [24]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorflow_log_dir, histogram_freq=0,  
          write_graph=False, write_images=False)

In [25]:
vgg19.fit(x_train, y_train, validation_split=VAL_SPLIT, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[tensorboard_callback], verbose=1)

Epoch 1/10
50000/50000 [==============================] - 53s 1ms/step - loss: 0.8848 - acc: 0.7031
Epoch 2/10
50000/50000 [==============================] - 48s 962us/step - loss: 0.6744 - acc: 0.7739
Epoch 3/10
50000/50000 [==============================] - 48s 967us/step - loss: 0.5377 - acc: 0.8193
Epoch 4/10
50000/50000 [==============================] - 49s 971us/step - loss: 0.4416 - acc: 0.8523
Epoch 5/10
50000/50000 [==============================] - 49s 970us/step - loss: 0.3578 - acc: 0.8794
Epoch 6/10
50000/50000 [==============================] - 49s 974us/step - loss: 0.2863 - acc: 0.9025
Epoch 7/10
50000/50000 [==============================] - 49s 973us/step - loss: 0.2294 - acc: 0.9225
Epoch 8/10
50000/50000 [==============================] - 49s 971us/step - loss: 0.1836 - acc: 0.9396
Epoch 9/10
50000/50000 [==============================] - 49s 971us/step - loss: 0.1542 - acc: 0.9502
Epoch 10/10
50000/50000 [==============================] - 49s 971us/step - loss: 0.

In [27]:
vgg19.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)

10000/10000 [==============================] - 3s 317us/step


[0.8394882464408875, 0.7821]